In [2]:
from util.ner_machine_learning import extract_embeddings_as_features_and_gold, \
    extract_features_and_labels, create_classifier
from gensim.models import KeyedVectors
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
import pandas as pd

#conll filepath
train_path = '../data/conll2003.train.conll'
dev_path = '../data/conll2003.dev.conll'

#word2vec binary filepath
google_news = '../data/GoogleNews-vectors-negative300.bin'

In [12]:
# Load the training data
data = pd.read_csv(train_path, delimiter='\t', quotechar='|', header=None)
data = data.rename({0: 'token', 1: 'pos', 2: 'chunk_tag', 3: 'target'}, axis=1)
# [row, column]
X_train = data[['token', 'pos', 'chunk_tag']]
Y_train = data['target']
#print(Y_train)

# Load the dev data
data = pd.read_csv(dev_path, delimiter='\t', quotechar='|', header=None)
data = data.rename({0: 'token', 1: 'pos', 2: 'chunk_tag', 3: 'target'}, axis=1)
X_dev = data[['token', 'pos', 'chunk_tag']]
Y_dev = data['target']

In [11]:
#load the binary model
wv_from_bin = KeyedVectors.load_word2vec_format(google_news, binary=True)

In [13]:
def word2vec(word):
    if word in wv_from_bin:
        result = wv_from_bin[word]
    else:
        result = [0] * 300
    return result


X_train = list(X_train['token'].apply(word2vec))
X_dev = list(X_dev['token'].apply(word2vec))

In [17]:
# Combining token embedding and one hot encoder into one vector
# ohe2 = OneHotEncoder(sparse=False)
# X_train_ohe = ohe2.fit_transform(X_train[['pos', 'chunk_tag']])
# ohe_word_embedding = pd.concat(
#     [pd.DataFrame(X_train_ohe), X_train['token_embedding']], axis=1)
print(len(X_train[0]))

300


In [18]:
model = LogisticRegression(max_iter = 100000)
model.fit(X_train,Y_train)
Y_pred = model.predict(X_dev)
logreg_rpt = classification_report(Y_dev, Y_pred, digits = 3)
print(logreg_rpt)

              precision    recall  f1-score   support

       B-LOC      0.831     0.825     0.828      1837
      B-MISC      0.803     0.704     0.750       922
       B-ORG      0.688     0.660     0.674      1341
       B-PER      0.781     0.710     0.744      1842
       I-LOC      0.596     0.545     0.569       257
      I-MISC      0.720     0.439     0.546       346
       I-ORG      0.586     0.414     0.485       751
       I-PER      0.617     0.578     0.597      1307
           O      0.977     0.995     0.986     42759

    accuracy                          0.940     51362
   macro avg      0.733     0.652     0.687     51362
weighted avg      0.936     0.940     0.937     51362

